In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster, HeatMap, HeatMapWithTime, FastMarkerCluster
import ipywidgets as widgets
import geopandas as gpd
import fiona
import json
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
from IPython.display import HTML, display
from collections import defaultdict
from shapely.geometry import Point
pd.set_option('display.max_rows', 999)

In [3]:
tenn_eq_geo_map = pd.read_csv('../Data/tenn_eq_geo_map.csv')

In [5]:
tenn_eq_geo_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052 entries, 0 to 1051
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   time             1052 non-null   object 
 1   latitude         1052 non-null   float64
 2   longitude        1052 non-null   float64
 3   depth            1049 non-null   float64
 4   mag              1052 non-null   float64
 5   magType          1052 non-null   object 
 6   nst              1015 non-null   float64
 7   gap              1015 non-null   float64
 8   dmin             959 non-null    float64
 9   rms              1015 non-null   float64
 10  net              1052 non-null   object 
 11  id               1052 non-null   object 
 12  updated          1052 non-null   object 
 13  place            1052 non-null   object 
 14  type             1052 non-null   object 
 15  horizontalError  1026 non-null   float64
 16  depthError       1025 non-null   float64
 17  magError      

In [4]:
tenn_eq_geo_map.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,index_right,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,county_name,tdot_region,tdot_code
0,2020-10-19T20:19:38.490Z,36.224000,-82.817167,17.27,2.31,md,8.0,70.0,0.17510,0.12,...,23,0500000US47059,47,59,Greene,County,622.165,Greene,1,30
1,2020-06-21T12:30:29.170Z,36.070333,-82.737167,6.12,2.47,md,26.0,67.0,0.05083,0.29,...,23,0500000US47059,47,59,Greene,County,622.165,Greene,1,30
2,2019-09-16T02:45:34.740Z,36.205167,-82.921167,4.94,2.00,md,5.0,133.0,0.21610,0.10,...,23,0500000US47059,47,59,Greene,County,622.165,Greene,1,30
3,2019-08-18T07:25:24.620Z,36.215500,-82.847333,5.55,2.16,md,8.0,86.0,0.18680,0.24,...,23,0500000US47059,47,59,Greene,County,622.165,Greene,1,30
4,2015-01-08T01:09:36.130Z,36.205667,-82.873500,3.12,2.38,md,18.0,60.0,0.20140,0.24,...,23,0500000US47059,47,59,Greene,County,622.165,Greene,1,30


In [6]:
eq_by_year_mag = tenn_eq_geo_map[['time', 'mag', 'tdot_region', 'position', 'geometry','county_name' ]]

In [7]:
eq_by_year_mag

,time,mag,tdot_region,position,geometry,county_name
0,2020-10-19T20:19:38.490Z,2.31,1,POINT (-82.8171667 36.224),POINT (-82.8171667 36.224),Greene
1,2020-06-21T12:30:29.170Z,2.47,1,POINT (-82.7371667 36.0703333),POINT (-82.7371667 36.0703333),Greene
2,2019-09-16T02:45:34.740Z,2.00,1,POINT (-82.9211667 36.2051667),POINT (-82.9211667 36.2051667),Greene
3,2019-08-18T07:25:24.620Z,2.16,1,POINT (-82.8473333 36.2155),POINT (-82.8473333 36.2155),Greene
4,2015-01-08T01:09:36.130Z,2.38,1,POINT (-82.87350000000001 36.2056667),POINT (-82.87350000000001 36.2056667),Greene
...,...,...,...,...,...,...
1047,1980-12-02T08:59:29.700Z,3.80,4,POINT (-89.429 36.175),POINT (-89.429 36.175),Dyer
1048,1962-07-23T06:05:16.300Z,3.39,4,POINT (-89.399 36.044),POINT (-89.399 36.044),Dyer
1049,1955-03-29T09:02:40.000Z,3.90,4,POINT (-89.5 36),POINT (-89.5 36),Dyer
1050,1918-10-16T02:15:00.000Z,4.50,4,POINT (-89.2 36),POINT (-89.2 36),Dyer


In [8]:
eq_by_year_mag['year'] = pd.DatetimeIndex(eq_by_year_mag['time']).year
#df['year'] = pd.DatetimeIndex(df['birth_date']).year

<ipython-input-8-4d669014cbf4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eq_by_year_mag['year'] = pd.DatetimeIndex(eq_by_year_mag['time']).year


In [9]:
eq_by_year_mag

,time,mag,tdot_region,position,geometry,county_name,year
0,2020-10-19T20:19:38.490Z,2.31,1,POINT (-82.8171667 36.224),POINT (-82.8171667 36.224),Greene,2020
1,2020-06-21T12:30:29.170Z,2.47,1,POINT (-82.7371667 36.0703333),POINT (-82.7371667 36.0703333),Greene,2020
2,2019-09-16T02:45:34.740Z,2.00,1,POINT (-82.9211667 36.2051667),POINT (-82.9211667 36.2051667),Greene,2019
3,2019-08-18T07:25:24.620Z,2.16,1,POINT (-82.8473333 36.2155),POINT (-82.8473333 36.2155),Greene,2019
4,2015-01-08T01:09:36.130Z,2.38,1,POINT (-82.87350000000001 36.2056667),POINT (-82.87350000000001 36.2056667),Greene,2015
...,...,...,...,...,...,...,...
1047,1980-12-02T08:59:29.700Z,3.80,4,POINT (-89.429 36.175),POINT (-89.429 36.175),Dyer,1980
1048,1962-07-23T06:05:16.300Z,3.39,4,POINT (-89.399 36.044),POINT (-89.399 36.044),Dyer,1962
1049,1955-03-29T09:02:40.000Z,3.90,4,POINT (-89.5 36),POINT (-89.5 36),Dyer,1955
1050,1918-10-16T02:15:00.000Z,4.50,4,POINT (-89.2 36),POINT (-89.2 36),Dyer,1918


In [ ]:
eq_by_year_mag.plot(year, )